In [14]:
import pandas as pd
import numpy as np
import re

In [13]:
def importExcel():

  xls = pd.ExcelFile(r'/content/sample_data/BaruchSchedule-2022-10-18.xlsx')

  df1 = pd.read_excel(xls, 'Grid2')
  df1.drop('Date', axis=1, inplace=True)

  dbt = pd.read_excel(xls, 'Working')

  dbt.drop('Mtg Start', axis=1, inplace=True)
  dbt.drop('Mtg End', axis=1, inplace=True)

  dbt['Days'] = dbt['Days'].fillna('Z')

  db = pd.DataFrame(columns = ['Subject','Catalog','Section','Tot Enrl','Days','Time'])

  for index in dbt.index:
    d = dbt['Days'][index]
    days = re.findall('[A-Z][^A-Z]*', d)

    for x in days:
      db.loc[len(db.index)]= [dbt['Subject'][index], dbt['Catalog'][index], dbt['Section'][index], dbt['Tot Enrl'][index], x, dbt['Time'][index]]

  return db, df1

def condition1(C1):

  if C1[0] == '0':
    return 0
  
  else:
    sub = [C1[0]]
    intCat = int(C1[1])
    cat = [intCat]

    if C1[2] == 'D':
      time = [C1[2]]
    elif C1[2] == 'E':
      time = [C1[2]]
    else:
      time = ['D','E']

    dfC1 = db.loc[(db['Subject'].isin(sub)) & (db['Catalog'].isin(cat)) & (db['Time'].isin(time))]

    total = dfC1['Tot Enrl'].sum()
    return total

def condition2(C2):

  TotalC2 = 0
  TotalC2 = 0

  for i in C2:

    if C2[0] == '0':
      continue
    else:
      dfC2 = db[db['Section'].str.startswith(i)]
      
      totalC2 = dfC2['Tot Enrl'].sum()
      TotalC2 = TotalC2 + totalC2
  
  return TotalC2

def condition34(C3, C4):

  if C3[0] == '0' and C4[0] == '0':
    return 0

  elif len(C4) == 1:
    dfC4 = db.loc[(db['Days'].isin(C4))]

  else:
    t1 = [C4[0]]
    t2 = [C4[1]]

    dfC4 = db.loc[(db['Days'].isin(C4))]

  totalC3 = 0
  TotalC3 = 0

  for i in C3:

    if C3[0] == '0':
      continue

    else:
      dfC3 = dfC4[dfC4['Section'].str.startswith(i)]

      totalC3 = dfC3['Tot Enrl'].sum()
      TotalC3 = TotalC3 + totalC3
  
  return TotalC3

def exportExcel(totalEnrollment):

  xls1 = pd.ExcelFile(r'/content/sample_data/BaruchSchedule-2022-10-18.xlsx')

  ds = pd.read_excel(xls1, 'Grid2')
  ds = ds.reset_index(drop=True)
  ds = ds.set_index('Date')

  header = list(ds)

  result  = pd.DataFrame(columns=header)

  index_list = list(ds.index.values)

  result['Date'] = index_list
  result.set_index('Date', inplace=True)

  result[header[0]] = totalEnrollment[0::6]
  result[header[1]] = totalEnrollment[1::6]
  result[header[2]] = totalEnrollment[2::6]
  result[header[3]] = totalEnrollment[3::6]
  result[header[4]] = totalEnrollment[4::6]
  result[header[5]] = totalEnrollment[5::6]

  result.to_excel('result2.xlsx', index = True)


In [15]:
if __name__ == "__main__":
  db, df1 = importExcel()

  totalEnrollment = []

  for i in range(df1.shape[0]): 
    for j in range(df1.shape[1]):
      condition = df1.iat[i, j]

      conditionList = condition.split()
      
      c1 = conditionList[0]
      c2 = conditionList[1]
      c3 = conditionList[2]
      c4 = conditionList[3]

      C1 = re.split(r'[^a-zA-Z0-9\s]', c1)
      C1 = [x for x in C1 if x != '']
      C1.pop(0)

      C2 = re.split(r'[^a-zA-Z0-9\s]', c2)
      C2 = [x for x in C2 if x != '']
      C2.pop(0)

      C3 = re.split(r'[^a-zA-Z0-9\s]', c3)
      C3 = [x for x in C3 if x != '']
      C3.pop(0)

      C4 = re.split(r'[^a-zA-Z0-9\s]', c4)
      C4 = [x for x in C4 if x != '']
      C4.pop(0)
   
      #FOR C1
      
      subListC1 = [C1[n:n+3] for n in range(0, len(C1), 3)]

      totalC1 = 0
      TotalC1 = 0

      for n in subListC1:
        totalC1 = condition1(n)
        TotalC1 = TotalC1 + totalC1

      #FOR C2

      TotalC2 = condition2(C2)

      #FOR CONDITION 3 AND 4

      TotalC3 = condition34(C3, C4)

      totalEnrollment.append(TotalC1+TotalC2+TotalC3)

exportExcel(totalEnrollment)
